In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterGrid

import bcnn
from bcnn.network import BinaryClassifier

# Generate data

$5 * 10^4$ pairs of random numbers from range $[-1, 1]$

In [2]:
def generate_data(size: int):
    data = np.random.uniform(low=0, high=1, size=(2, size))
    data = data * np.random.choice([-1, 1], size=(2, size))
    labels = (np.sign(data[0]) == np.sign(data[1])).astype(int)
    return data, labels

In [3]:
train_data, train_labels = generate_data(50_000)
train_data.shape, train_labels.shape

((2, 50000), (50000,))

In [4]:
test_data, test_labels = generate_data(10_000)
test_data.shape, test_labels.shape

((2, 10000), (10000,))

# Instantiate and fit the models

Model to the detect whether two numbers from range $[-1, 1]$ have the same sign

In [5]:
INPUT_SIZE = 2
NEURONS = [4]
METRIC = bcnn.metrics.Accuracy()
LOSS = bcnn.losses.BinaryCrossentropy()

params_dict = {
    'activation': [
        bcnn.activation.ReLU, 
        bcnn.activation.Sigmoid
    ],
    'normalizer': [
        None, 
        bcnn.normalizers.L1, 
        bcnn.normalizers.L2
    ],
    'learning_rate': [0.001, 0.005, 0.01]
}
params_dict

{'activation': [bcnn.activation.Relu, bcnn.activation.Sigmoid],
 'normalizer': [None, bcnn.normalizers.L1, bcnn.normalizers.L2],
 'learning_rate': [0.001, 0.005, 0.01]}

In [6]:
param_grid = pd.DataFrame(ParameterGrid(params_dict))
param_grid

,activation,learning_rate,normalizer
0,<class 'bcnn.activation.Relu'>,0.001,None
1,<class 'bcnn.activation.Relu'>,0.001,<class 'bcnn.normalizers.L1'>
2,<class 'bcnn.activation.Relu'>,0.001,<class 'bcnn.normalizers.L2'>
3,<class 'bcnn.activation.Relu'>,0.005,None
4,<class 'bcnn.activation.Relu'>,0.005,<class 'bcnn.normalizers.L1'>
5,<class 'bcnn.activation.Relu'>,0.005,<class 'bcnn.normalizers.L2'>
6,<class 'bcnn.activation.Relu'>,0.010,None
7,<class 'bcnn.activation.Relu'>,0.010,<class 'bcnn.normalizers.L1'>
8,<class 'bcnn.activation.Relu'>,0.010,<class 'bcnn.normalizers.L2'>
9,<class 'bcnn.activation.Sigmoid'>,0.001,None


In [7]:
model_dict = {
    'model': [],
    'loss': [],
    'metric': []
}

for i in param_grid.index:
    # instantiate the model
    norm = None if param_grid.loc[i, 'normalizer'] is None else param_grid.loc[i, 'normalizer']()
    model = BinaryClassifier(
        input_size=INPUT_SIZE,
        neurons=NEURONS,
        activation=param_grid.loc[i, 'activation'](),
        metric=METRIC,
        loss=LOSS,
        normalizer=norm
    )

    # fit the model
    model.fit(
        X=train_data, 
        Y=train_labels, 
        learning_rate=param_grid.loc[i, 'learning_rate'],
        verbose=False
    )

    # evaluate the model
    loss, metric = model.evaluate(test_data, test_labels)
    model_dict['model'].append(model)
    model_dict['loss'].append(loss)
    model_dict['metric'].append(metric)

    print(f"{i}: {loss = }, {metric = }") 

0: loss = 0.6812142050880211, metric = 0.563
1: loss = 1.850949374379479, metric = 0.5024
2: loss = 0.6921450632995781, metric = 0.5098
3: loss = 0.9213588270955595, metric = 0.6126
4: loss = 0.6351293286553932, metric = 0.646
5: loss = 0.6929527275993362, metric = 0.5167
6: loss = 0.23433276243310228, metric = 0.9292
7: loss = 0.5883518678536, metric = 0.7191
8: loss = 0.6915279275004957, metric = 0.504
9: loss = 0.7198526103813999, metric = 0.504
10: loss = 0.7416519665721142, metric = 0.496
11: loss = 0.7187094224315633, metric = 0.504
12: loss = 0.7109002448830125, metric = 0.4687
13: loss = 0.6987597505078047, metric = 0.595
14: loss = 0.6944959872887257, metric = 0.496
15: loss = 0.697549059909455, metric = 0.4821
16: loss = 0.6969947808676603, metric = 0.4636
17: loss = 0.6930986054628112, metric = 0.504


In [8]:
model_grid = pd.DataFrame(model_dict)
model_grid

,model,loss,metric
0,<bcnn.network.BinaryClassifier object at 0x000...,0.681214,0.5630
1,<bcnn.network.BinaryClassifier object at 0x000...,1.850949,0.5024
2,<bcnn.network.BinaryClassifier object at 0x000...,0.692145,0.5098
3,<bcnn.network.BinaryClassifier object at 0x000...,0.921359,0.6126
4,<bcnn.network.BinaryClassifier object at 0x000...,0.635129,0.6460
5,<bcnn.network.BinaryClassifier object at 0x000...,0.692953,0.5167
6,<bcnn.network.BinaryClassifier object at 0x000...,0.234333,0.9292
7,<bcnn.network.BinaryClassifier object at 0x000...,0.588352,0.7191
8,<bcnn.network.BinaryClassifier object at 0x000...,0.691528,0.5040
9,<bcnn.network.BinaryClassifier object at 0x000...,0.719853,0.5040


In [9]:
best_model = model_grid.loc[model_grid['metric'].idxmax()]
best_model.model.summary()
best_model.loss, best_model.metric

Number of layers: 3
Number of neurons per layer:
	1: 2
	2: 4
	3: 1
Activation: relu
Metric: accuracy
Loss: binary_crossentropy
Normalization: none


(0.23433276243310228, 0.9292)